# 二創 Scenario

## 1. Initialize agent

In [ ]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

API_KEY = "bob_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)

agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST, 
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

## 2. DM-MUSIC did exchange

### Build connection

In [ ]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)

In [ ]:
##############
# Invitation #
##############

invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '5f71807a-8c5b-4e8d-8413-1ad01a96d169', 'serviceEndpoint': 'http://192.168.65.3:8020', 'recipientKeys': ['gqoZHV5AEYdr7qAqckN4wMT9Mm3HzSbUrrxr7XvaN1P'], 'label': 'Alice'}


In [ ]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
alice_id = response["connection_id"]

### > Break here - switch to DM accept request <

### Send DID to DM

In [ ]:
# Print connection list
connection = await agent_controller.connections.get_connection(alice_id)
print("Alice AGENT CONNECTION")
print(connection)
print("State:", connection["state"])
music_connection = connection["connection_id"]
print(music_connection)

In [ ]:
my_did = await agent_controller.wallet.get_public_did()
my_did = my_did['result']['did']
print("my DID", my_did)
dm_did = 'None'

In [ ]:
def messages_handler(payload):
    global dm_did
    connection_id = payload["connection_id"]
    print("Handle message", payload, connection_id)
    dm_did = payload['content']
    print('DM DID', music_did)


message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener], defaults=True)

In [ ]:
response = await agent_controller.messaging.send_message(music_connection, 'music_'+my_did)
print(response)

In [ ]:
print(dm_did)
print(my_did)

## 3. Verify the song with SP - MUSIC

### Basic setup - existence check and listener

In [ ]:
##############
# Invitation #
##############

invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'f333199a-9381-4fa5-b23d-34c0adbcb2d1', 'label': 'Carol', 'serviceEndpoint': 'http://192.168.65.3:8080', 'recipientKeys': ['EJmRUbfL5tabRD5EDmDnCo3tVPGUewGxoVmq8jbwLhFn']}

In [ ]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
alice_id = response["connection_id"]

In [ ]:
# Song exists?
credential_id = "My Credential"
credential = await agent_controller.credentials.get_by_id(credential_id)
print(credential)

In [ ]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def proof_handler(payload):
    global presentation_exchange_id
    print("Handle present proof")
    role = payload["role"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print(f"Role {role}, Exchange {pres_ex_id} in state {state}")
    if state == "presentation_received":
     
        presentation_exchange_id = pres_ex_id 
        print(presentation_exchange_id)
proof_listener = {
    "topic": "present_proof",
    "handler": proof_handler
}
agent_controller.register_listeners([proof_listener], defaults=True)

### > Break: Go to SP verifier for getting the song it wants to recreate <

### Fetch proof request records

In [ ]:
from datetime import datetime
response = await agent_controller.proofs.get_records()
print(response)

format = "%Y-%m-%d %H:%M:%S.%fZ"


response["results"] = list(filter(lambda r: r["state"] == "request_received", response["results"]))
response["results"] = sorted(
        response["results"],
        key=lambda c: datetime.strptime(c["created_at"], format),
        reverse=True,
    )
print(response["results"][0]["created_at"])
print(response["results"][0]["state"])
print('\n')
state = response['results'][0]["state"]
presentation_exchange_id = response['results'][-1]['presentation_exchange_id']
presentation_request = response['results'][-1]['presentation_request']

print('Presentation Exchange ID\n')
print(response['results'][0]['presentation_exchange_id'])
print('Presentation Request Object\n')
print(response['results'][0]['presentation_request'])
print('Requested Attributes\n')
print(response['results'][0]['presentation_request']['requested_attributes'])
requested_attribs = response['results'][-1]['presentation_request']['requested_attributes']
print('Requested Predicates\n')
print(response['results'][0]['presentation_request']['requested_predicates'])
requested_predicates = response['results'][-1]['presentation_request']['requested_predicates']

In [ ]:
if state == "request_received":
    print(
    "Received Request -> Query for credentials in the wallet that satisfy the proof request")

# include self-attested attributes (not included in credentials)
credentials_by_reft = {}
revealed = {}
self_attested = {}
predicates = {}

# select credentials to provide for the proof
credentials = await agent_controller.proofs.get_presentation_credentials(presentation_exchange_id)
print(credentials)

if credentials:
    for row in sorted(
        credentials,
        key=lambda c: dict(c["cred_info"]["attrs"]),
        reverse=True,
    ):
        for referent in row["presentation_referents"]:
            if referent not in credentials_by_reft:
                credentials_by_reft[referent] = row

for referent in presentation_request["requested_attributes"]:
    if referent in credentials_by_reft:
        revealed[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ],
            "revealed": True,
        }
    else:
        self_attested[referent] = "South Africa"

for referent in presentation_request["requested_predicates"]:
    if referent in credentials_by_reft:
        predicates[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ]
        }

print("\nGenerate the proof")
proof = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}
print(proof)
print("\nXXX")
print(predicates)
print(revealed)
print(self_attested)

### Send proof back!

In [ ]:
response = await agent_controller.proofs.send_presentation(presentation_exchange_id, proof)
print(response)

## 5. Check the VC of SP, if they want to publish their recreation - MUSIC is the Verifier

Scenario:  
Some SP will contact the MUSIC platform for **recreation publishing**.    
In this case, the platform will check the SP's recreation-VC to see that if the SP has the right to write the VC of ownership of recreation on their platform.    
If the VC has the correct recreation-VC, then it can post their recreation artpiece on the platform with an ownership-VC that has 
```{ author: recreator, origin: creator, ...}```


### Send proof request

In [ ]:
try:
    response = await agent_controller.connections.get_connections()
    results = response['results']
    print("Results : ", results)
    print('\n')
    if len(results) > 0:
        connections = response['results']
        print("Connection :", connections)
        for connection in connections:
            if connection['state'] == 'active' and connection["their_label"] == "Carol":
                connection_id = connection["connection_id"]
                print("\nActive Connection ID : ", connection_id)
            else:
                print("\nNo active connection found - wait a bit and execute again")
    else:
        print("You must create a connection")
except ConnectionRefusedError as e:
    print(repr(e))

In [ ]:
response = await agent_controller.wallet.get_public_did()
issuer_did = dm_did
print("issuer: dm, has ", dm_did)

print("Request proof of Name and Age range from Bob")
#Set some variables

exchange_tracing = False

#Enable this to ask for attributes to identity a user
req_attrs = [
    {"name": "target", "restrictions": [{"issuer_did": issuer_did}]},
    {"name": "time", "restrictions": [{"issuer_did": issuer_did}]},
    {"name": "origin", "restrictions": [{"issuer_did": issuer_did}]},
    {"name": "owner", "restrictions": [{"issuer_did": issuer_did}]},
]


#Set predicates for Zero Knowledge Proofs
req_preds = []

indy_proof_request = {
    "name": "Proof of Personal Information",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
        f"0_{req_pred['name']}_GE_uuid":
        req_pred for req_pred in req_preds
    },
}

#proof_request = indy_proof_request
exchange_tracing_id = exchange_tracing
proof_request_web_request = {
    "connection_id": connection_id,
    "proof_request": indy_proof_request,
    "trace": exchange_tracing,
}

In [ ]:
response = await agent_controller.proofs.send_request(proof_request_web_request)
print(response)
presentation_exchange_id = response['presentation_exchange_id']
print("\n")
print(presentation_exchange_id)

### >Break: Finish sending proof, check MUSIC for the proof request!<

### Verify the result back

### Check the song1

In [ ]:
verify = await agent_controller.proofs.verify_presentation(presentation_exchange_id)
print(verify)

In [ ]:
## TODO: if verify:
target_song = verify["presentation"]["requested_proof"]["revealed_attrs"]["0_target_uuid"]["raw"]
owner = verify["presentation"]["requested_proof"]["revealed_attrs"]["0_owner_uuid"]["raw"]
issuer = verify["presentation_request"]["requested_attributes"]["0_target_uuid"]["restrictions"][0]["issuer_did"]
print(target_song)
print(owner)
print(issuer)

In [ ]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")

In [ ]:
# Request credential from issuer
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

In [ ]:
record = await agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

In [ ]:
all_credential = await agent_controller.credentials.get_all()
print(all_credential)
ownership_credential = list(filter(lambda r: r["referent"] == "My Credential", all_credential["results"]))
print(ownership_credential)
recreation_credential = list(filter(lambda r: r["attrs"]["music"] == target_song and r["attrs"]["owner"] == issuer, ownership_credential))
print(recreation_credential)

In [ ]:
# Store credential
print(attributes)
print(list(filter(lambda r: r["name"] == "owner" and r["value"] == owner, attributes)))
ver = len(list(filter(lambda r: r["name"] == "owner" and r["value"] == owner, attributes))) > 0
print(ver)

if ver and len(recreation_credential) > 0:
    response = await agent_controller.issuer.store_credential(cred_ex_id, "My Credential")
    state = response['state']
    role = response['role']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

# Terminate Agent

In [ ]:
response = await agent_controller.terminate()
print(response)